# Atmospheric retrieval with petitRADTRANS

This is a tutorial for atmospheric retrieval with [petitRADTRANS](https://petitradtrans.readthedocs.io/) for which we will use NIR spectra and photometry of the planetary-mass companion [ROXs 42 Bb](https://ui.adsabs.harvard.edu/abs/2014ApJ...780L..30C/abstract). Free retrievals are computationally expensive due to the high number of parameter dimensions and the scattering radiative transfer that is important for cloudy objects (see [Mollière et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A.131M/abstract)). Similar [FitModel](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.fit_model.FitModel), the nested sampling with the free retrievals uses multiprocessing so it recommended to run such retrievals on a cluster. Before starting, ``petitRADTRANS`` should be manually installed together with the line and continuum opacities.

When running a retrieval, it is important to comment out any of the functions that access the [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) because writing to the HDF5 database is not possible with multiprocessing. Therefore, the companion data should first be added to the database, then [AtmosphericRetrieval](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.retrieval.AtmosphericRetrieval) and [run_multinest](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.retrieval.AtmosphericRetrieval.run_multinest) can be executed (e.g. on a cluster), and finally the results can be extracted and plotted while commenting out the retrieval part.

## Import *species* and setup database

We start by adding the library path of ``MultiNest`` to the ``DYLD_LIBRARY_PATH`` environment variable such that ``PyMultiNest`` can find this library.

In [ ]:
import os
os.environ['DYLD_LIBRARY_PATH'] = '/Users/tomasstolker/applications/MultiNest/lib'

Next, we import the `species` toolkit and initialize the HDF5 database in the working by running the [SpeciesInit](https://species.readthedocs.io/en/latest/species.core.html#species.core.init.SpeciesInit) class. This will also create a [configuration file](https://species.readthedocs.io/en/latest/configuration.html) in case it was not yet present in the working folder.

In [ ]:
import species
species.SpeciesInit()

We also create an instance of [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) which is used for accessing the HDF5 database.

In [ ]:
database = species.Database()

We will now add the photometric data of [ROXs 42 Bb](https://exoplanets.nasa.gov/exoplanet-catalog/7139/roxs-42-b-b/) that are [available in species](https://github.com/tomasstolker/species/blob/master/species/data/companions.py). This is automatically done with the [add_companion](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_companion) method. Alternatively, the [add_object](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_object) method can be used to manually add magnitudes and spectra (see below). Running `add_companion` will also download the filter profiles and a flux-calibrated spectrum of Vega in order to convert the magnitudes into fluxes.

In [ ]:
database.add_companion('ROXs 42 Bb')

The distance and magnitudes of ROXs 42 Bb have now been stored but we also want to add medium-resolution $JHK$-band spectra. This is done with the [add_object](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.add_object) method by providing a dictionary as argument of `spectrum`. The tuple of each spectrum name contains the filename with the spectrum, optionally the covariance matrix, and the spectral resolution (that is used for smoothing the model spectra). We need to make sure that the argument of `object_name` is the same name as with [add_companion](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.add_companion) such that data are added to the correct database group.

In [ ]:
database.add_object(object_name='ROXs 42 Bb',
                    distance=None,
                    app_mag=None,
                    spectrum={'NIFS': ('roxs42bb_nifs_j_calib.dat', None, 5000.),
                              'OSIRIS': ('roxs42bb_osiris_h_calib.dat', None, 3000.),
                              'SINFONI': ('roxs42bb_sinfoni_k_calib.dat', None, 1500.)},
                    deredden=None)

## Atmospheric retrieval of abundances, clouds, and P-T structure

In [ ]:
retrieve = species.AtmosphericRetrieval(object_name='ROXs 42 Bb',
                                        line_species=['CO_all_iso', 'H2O', 'CH4', 'NH3', 'CO2', 'Na', 'K', 'TiO', 'VO', 'FeH', 'H2S'],
                                        cloud_species=['MgSiO3(c)_cd', 'Fe(c)_cd', 'Al2O3(c)_cd'],
                                        scattering=True,
                                        output_folder='multinest',
                                        wavel_range=(1.1, 2.46),
                                        inc_spec=['NIFS', 'OSIRIS', 'SINFONI'],
                                        inc_phot=False,
                                        pressure_grid='smaller',
                                        weights=None)

In [ ]:
retrieve.run_multinest(bounds={'logg': (2., 5.5),
                               'c_o_ratio': (0.1, 1.),
                               'metallicity': (-3., 3.),
                               'radius': (1., 5.),
                               'ism_ext': (1.7, 1.7),
                               'OSIRIS': ((0.8, 1.2), None, None),
                               'fsed': (0., 8.),
                               'log_kzz': (4., 12.),
                               'log_tau_cloud': (-2., 1.),
                               'fe_mgsio3_ratio': (-2., 2.),
                               'al2o3_mgsio3_ratio': (-2., 2.)},
                       chemistry='equilibrium',
                       quenching=None,
                       pt_profile='molliere',
                       n_live_points=1000,
                       resume=True,
                       plotting=False,
                       pt_smooth=0.)

## Nested sampling output folder

The output data from the nested sampling with ``MultiNest`` is stored in the ``output_folder``. We can use the [add_retrieval](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_retrieval) method of [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) to store the posterior samples and relevant attributes in the HDF5 database. The argument of ``tag`` is used as name tag in the database. It is also possible to calculate $T_\mathrm{eff}$ for each sample but this takes a long time because each spectrum needs to be calculated over a broad wavelength range.

In [ ]:
database.add_retrieval(tag='roxs42bb',
                       output_folder='multinest',
                       inc_teff=False)

Instead, we use the [get_retrieval_teff](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_retrieval) to estimate $T_\mathrm{eff}$ from a small number of samples. The value is stored in the database as attribute of the ``tag`` group.

In [ ]:
database.get_retrieval_teff(tag='roxs42bb',
                            random=30)

## Plotting the posterior distributions

We can now read the posterior samples from the database and use the plot functionalities to visualize the results. Let's first plot the marginalized posterior distributions by using the [corner.py](https://corner.readthedocs.io) package. The plot is created with the [plot_posterior](https://species.readthedocs.io/en/latest/species.plot.html#species.plot.plot_mcmc.plot_posterior) function. Since there are many free parameters, we will leave out those for the P-T profile by setting `inc_pt_param=False`.

In [ ]:
species.plot_posterior(tag='roxs42bb',
                       offset=(-0.3, -0.35),
                       vmr=False,
                       inc_mass=False,
                       inc_pt_param=False,
                       output='posterior.png')

Let's have a look at the corner plot!

In [ ]:
from IPython.display import Image
Image('posterior.png')

## ReadRadtrans and random spectra

In order to post-process the posterior samples, we need to recreate the ``Radtrans`` object of ``petitRADTRANS``. We will use the [get_retrieval_spectra](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.get_retrieval_spectra) method of [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) to create and instance of [ReadRadtrans](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_radtrans.ReadRadtrans) with the adopted parameters from the retrieval. The ``Radtrans`` object is stored as an attribute of ``ReadRadtrans`` and will be used by ``species`` but can typically be ignored by the user. The method also returns a list of random spectra (30 in the example below) that have been recalculated at a resolving power of $R = 2000$. Each of the model spectra is stored in a [ModelBox](https://species.readthedocs.io/en/latest/species.core.html#species.core.box.ModelBox).

In [ ]:
samples, radtrans = database.get_retrieval_spectra(tag='roxs42bb',
                                                   random=30,
                                                   wavel_range=(0.5, 6.),
                                                   spec_res=2000.)

# Create plots of P-T profiles, opacities, and clouds

In [ ]:
species.plot_pt_profile(tag='roxs42bb',
                        random=100,
                        xlim=(0., 6000.),
                        offset=(-0.07, -0.14),
                        output='plot/pt_profile_grains.pdf',
                        radtrans=radtrans,
                        extra_axis='grains')

species.plot_opacities(tag='roxs42bb',
                       offset=(-0.1, -0.14),
                       output='plot/opacities.pdf',
                       radtrans=radtrans)

species.plot_clouds(tag='roxs42bb',
                    offset=(-0.1, -0.15),
                    output='plot/clouds.pdf',
                    radtrans=radtrans,
                    composition='MgSiO3')

## Read companion data, best-fit sample, and fit residuals

In [ ]:
best = database.get_probable_sample(tag='roxs42bb')

objectbox = database.get_object('ROXs 42 Bb',
                                inc_phot=False)

objectbox = species.update_spectra(objectbox, best)

residuals = species.get_residuals(datatype='model',
                                  spectrum='petitradtrans',
                                  parameters=best,
                                  objectbox=objectbox,
                                  inc_phot=False,
                                  inc_spec=True,
                                  radtrans=radtrans)

modelbox = radtrans.get_model(model_param=best,
                              spec_res=2000.,
                              plot_contribution='plot/contribution.pdf')

no_clouds = best.copy()
no_clouds['log_tau_cloud'] = -100.
model_no_clouds = radtrans.get_model(no_clouds)

## Plot the SED with data and models

In [ ]:
species.plot_spectrum(boxes=[samples, modelbox, model_no_clouds, objectbox],
                      filters=None,
                      plot_kwargs=[{'ls': '-', 'lw': 0.1, 'color': 'gray'},
                                   {'ls': '-', 'lw': 0.5, 'color': 'black'},
                                   {'ls': '--', 'lw': 0.3, 'color': 'black'},
                                   {'NIFS': {'marker': 'o', 'ms': 2., 'color': 'tab:green', 'ls': 'none', 'alpha': 0.2, 'mew': 0., 'label': 'NIFS'},
                                    'OSIRIS': {'marker': 'o', 'ms': 2., 'color': 'tab:blue', 'ls': 'none', 'alpha': 0.2, 'mew': 0., 'label': 'OSIRIS'},
                                    'SINFONI': {'marker': 'o', 'ms': 2., 'color': 'tab:orange', 'ls': 'none', 'alpha': 0.2, 'mew': 0., 'label': 'SINFONI'}}],
                      residuals=residuals,
                      xlim=(1.1, 2.5),
                      ylim=(0.15e-16, 1.15e-15),
                      ylim_res=(-5., 5.),
                      scale=('linear', 'linear'),
                      offset=(-0.6, -0.05),
                      figsize=(8, 2.5),
                      legend=[{'loc': 'upper right', 'fontsize': 8.}, {'loc': 'lower left', 'fontsize': 8.}],
                      output='plot/spectrum.pdf')

## Plot the SED over broader wavelength range

In [ ]:
modelbox = radtrans.get_model(model_param=best,
                              spec_res=200.,
                              plot_contribution='plot/contribution.pdf')

objectbox = database.get_object('ROXs 42 Bb',
                                inc_phot=True)

objectbox = species.update_spectra(objectbox, best)

for item in ['NIFS', 'OSIRIS', 'SINFONI']:
    objectbox.spectrum[item] = (objectbox.spectrum[item][0][::50, ],
                                objectbox.spectrum[item][1],
                                objectbox.spectrum[item][2],
                                objectbox.spectrum[item][3])

residuals = species.get_residuals(datatype='model',
                                  spectrum='petitradtrans',
                                  parameters=best,
                                  objectbox=objectbox,
                                  inc_phot=True,
                                  inc_spec=True,
                                  radtrans=radtrans)

synphot = species.multi_photometry(datatype='model',
                                   spectrum='petitradtrans',
                                   filters=objectbox.filters,
                                   parameters=best,
                                   radtrans=radtrans)

object_kwargs = {'NIFS': {'marker': 'o', 'ms': 2.5, 'color': 'tab:green', 'ls': 'none', 'alpha': 0.5, 'mew': 0., 'label': 'Gemini/NIFS'},
                 'OSIRIS': {'marker': 'o', 'ms': 2.5, 'color': 'tab:blue', 'ls': 'none', 'alpha': 0.5, 'mew': 0., 'label': 'Keck/OSIRIS'},
                 'SINFONI': {'marker': 'o', 'ms': 2.5, 'color': 'tab:orange', 'ls': 'none', 'alpha': 0.5, 'mew': 0., 'label': 'VLT/SINFONI'}}

for item in objectbox.magnitude:
    object_kwargs[item] = {'marker': 's', 'ms': 3.5, 'color': 'black'}

species.plot_spectrum(boxes=[samples, modelbox, objectbox, synphot],
                      filters=None,
                      plot_kwargs=[{'ls': '-', 'lw': 0.1, 'color': 'gray'},
                                   {'ls': '-', 'lw': 0.5, 'color': 'black'},
                                   object_kwargs,
                                   None],
                      residuals=residuals,
                      xlim=(0.8, 5.),
                      ylim=(0.15e-15, 1.9e-15),
                      ylim_res=(-7., 7.),
                      scale=('linear', 'linear'),
                      offset=(-0.6, -0.04),
                      figsize=(8, 2.5),
                      legend=[None, {'loc': 'upper right', 'fontsize': 10.}],
                      quantity='flux',
                      output='plot/full_sed.pdf')